In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [2]:
load_dotenv(dotenv_path=".env")
db = os.getenv("db_url")
engine = create_engine(db)

In [3]:
nyc = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.nyc", con=engine)
la = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.la", con=engine)
chicago = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.chicago", con=engine)
bay = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.bay", con=engine)
dallas = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.dallas", con=engine)
dc = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.dc", con=engine)
houston = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.houston", con=engine)
boston = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.boston", con=engine)
atlanta = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.atlanta", con=engine)
seattle = pd.read_sql("SELECT ur20, area, isoperi, solidity, rectan, pop_den, geom FROM public.seattle", con=engine)

In [4]:
nyc['city'] = 'nyc'
la['city'] = 'la'
chicago['city'] = 'chicago'
bay['city'] = 'bay'
dallas['city'] = 'dallas'
dc['city'] = 'dc'
houston['city'] = 'houston'
boston['city'] = 'boston'
atlanta['city'] = 'atlanta'
seattle['city'] = 'seattle'

In [5]:
combined = pd.concat([nyc, la, chicago, bay, dallas, dc, houston, boston, atlanta, seattle]).reset_index(drop=True)
filtered = combined.loc[combined['city']=='nyc']
filtered

,ur20,area,isoperi,solidity,rectan,pop_den,geom,city
0,U,16200.825,0.697,1.000,0.956,28616.003,0106000020E61000000100000001030000000100000005...,nyc
1,U,24292.633,0.609,1.000,0.949,8742.396,0106000020E61000000100000001030000000100000005...,nyc
2,U,19628.887,0.663,1.000,0.964,10555.762,0106000020E61000000100000001030000000100000005...,nyc
3,U,15393.900,0.561,1.000,0.979,7739.343,0106000020E61000000100000001030000000100000005...,nyc
4,U,14438.490,0.654,1.000,0.903,80365.333,0106000020E61000000100000001030000000100000005...,nyc
...,...,...,...,...,...,...,...,...
122510,U,1207.169,0.099,0.987,0.917,0.000,0106000020E6100000010000000103000000010000000A...,nyc
122511,U,26240.455,0.478,0.983,0.804,6415.748,0106000020E61000000100000001030000000100000007...,nyc
122512,U,26385.473,0.645,0.972,0.890,8638.202,0106000020E6100000010000000103000000010000000D...,nyc
122513,U,12150.579,0.616,1.000,0.979,9378.609,0106000020E61000000100000001030000000100000005...,nyc


In [6]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from shapely import wkb, wkt
from shapely.geometry import Polygon, MultiPolygon,LinearRing

def parse_geom(g):
    return wkb.loads(bytes.fromhex(g))

def _turning_angle(p_prev, p, p_next):
    v1 = p_prev - p
    v2 = p_next - p
    n1 = np.linalg.norm(v1) + 1e-12
    n2 = np.linalg.norm(v2) + 1e-12
    cosang = np.clip(np.dot(v1, v2) / (n1 * n2), -1.0, 1.0)
    return np.arccos(cosang)

def polygon_to_graph(poly, y_value=None):
    poly = wkb.loads(bytes.fromhex(poly))

    if isinstance(poly, MultiPolygon):
        poly = max(poly.geoms, key=lambda p: p.area)

    ring = LinearRing(poly.exterior.coords)
    coords = np.array(ring.coords[:-1], dtype=np.float32)
    n = coords.shape[0]
    if n < 3:
        return None

    centroid = coords.mean(axis=0, keepdims=True)
    coords0 = coords - centroid
    scale = np.sqrt((poly.area if poly.area > 0 else 1.0))
    coords0 = coords0 / (scale + 1e-12)

    prev_idx = (np.arange(n) - 1) % n
    next_idx = (np.arange(n) + 1) % n

    angles = np.array(
        [_turning_angle(coords0[prev_idx[i]], coords0[i], coords0[next_idx[i]]) for i in range(n)],
        dtype=np.float32
    )

    e_prev = np.linalg.norm(coords0 - coords0[prev_idx], axis=1).astype(np.float32)
    e_next = np.linalg.norm(coords0[next_idx] - coords0, axis=1).astype(np.float32)
    elen = np.stack([e_prev, e_next], axis=1)

    x = np.concatenate([coords0, angles[:, None], elen], axis=1)
    x = torch.from_numpy(x)

    src = torch.arange(n, dtype=torch.long)
    dst = torch.from_numpy(next_idx.astype(np.int64))
    edge_index = torch.stack([torch.cat([src, dst]), torch.cat([dst, src])], dim=0)

    data = Data(x=x, edge_index=edge_index)
    if y_value is not None:
        data.y = torch.tensor([float(y_value)], dtype=torch.float32)
    return data

c:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\libpyg.pyd
  import torch_geometric.typing
c:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\torch_scatter\_version_cuda.pyd
  import torch_geometric.typing
c:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\torch_geometric\__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: Could not load this library: C:\Users\kdgki\anaconda3\envs\geospatial\Lib\site-packages\torch_cluster\_version_cuda.pyd
  import torch_geometric.typing
c:\Users\k

In [7]:
filtered["graph"] = filtered["geom"].apply(polygon_to_graph)

C:\Users\kdgki\AppData\Local\Temp\ipykernel_37764\3561659828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["graph"] = filtered["geom"].apply(polygon_to_graph)


In [8]:
filtered

,ur20,area,isoperi,solidity,rectan,pop_den,geom,city,graph
0,U,16200.825,0.697,1.000,0.956,28616.003,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.2087, 0.7334, 1.6551, 1.41..."
1,U,24292.633,0.609,1.000,0.949,8742.396,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.0095, 0.8443, 1.7148, 1.66..."
2,U,19628.887,0.663,1.000,0.964,10555.762,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.1794, 0.7782, 1.6553, 1.51..."
3,U,15393.900,0.561,1.000,0.979,7739.343,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.9450, 0.2050, 1.5157, 0.54..."
4,U,14438.490,0.654,1.000,0.903,80365.333,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.8181, -0.3691, 1.3325, 0.67..."
...,...,...,...,...,...,...,...,...,...
122510,U,1207.169,0.099,0.987,0.917,0.000,0106000020E6100000010000000103000000010000000A...,nyc,"[(x, [tensor([-2.9138, -0.3084, 1.9168, 0.19..."
122511,U,26240.455,0.478,0.983,0.804,6415.748,0106000020E61000000100000001030000000100000007...,nyc,"[(x, [tensor([-1.0040, 0.0434, 3.1118, 0.41..."
122512,U,26385.473,0.645,0.972,0.890,8638.202,0106000020E6100000010000000103000000010000000D...,nyc,"[(x, [tensor([-0.4232, -0.0432, 2.8995, 0.11..."
122513,U,12150.579,0.616,1.000,0.979,9378.609,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.8584, 0.2414, 1.5201, 0.60..."


In [ ]:
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader

class GraphTabularDataset(Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.feature_cols = ["area", "isoperi", "solidity", "rectan"]
        self.target_col = "pop_den"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        g = row["graph"]
        tab = torch.tensor(row[self.feature_cols].to_numpy(dtype="float32"))
        city = torch.tensor(self.city_to_idx[str(row["city"])], dtype=torch.long)
        y = torch.tensor([float(row[self.target_col])], dtype=torch.float32)

        g = g.clone()
        g.tab = tab
        g.city = city
        g.y = y
        return g

dataset = GraphTabularDataset(filtered)
loader = DataLoader(dataset, batch_size=512, shuffle=True)

In [10]:
nodes = []
edges = []
for i in range(1000):
    g = dataset[i]
    nodes.append(g.num_nodes)
    edges.append(g.edge_index.size(1))

print(min(nodes), max(nodes), sum(nodes)/len(nodes))
print(min(edges), max(edges), sum(edges)/len(edges))

3 1524 38.706
6 3048 77.412


# Graph Training Model

In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset
from torch_geometric.nn import GCNConv, global_mean_pool

class GraphTabModel(nn.Module):
    def __init__(self, node_in_dim: int, tab_dim: int, num_cities: int, gnn_hidden: int = 64, city_emb_dim: int = 8, mlp_hidden: int = 64):
        super().__init__()
        self.conv1 = GCNConv(node_in_dim, gnn_hidden)
        self.conv2 = GCNConv(gnn_hidden, gnn_hidden)
        self.city_emb = nn.Embedding(num_cities, city_emb_dim)
        in_dim = gnn_hidden + tab_dim + city_emb_dim
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, mlp_hidden),
            nn.ReLU(),
            nn.Linear(mlp_hidden, 1),
        )

    def forward(self, batch):
        x = self.conv1(batch.x, batch.edge_index)
        x = torch.relu(x)
        x = self.conv2(x, batch.edge_index)
        x = torch.relu(x)

        g = global_mean_pool(x, batch.batch)

        tab = batch.tab
        if tab.dim() == 1:
            tab = tab.view(g.size(0), -1)

        city = batch.city
        if city.dim() == 0:
            city = city.view(1)
        if city.dim() == 1:
            city = self.city_emb(city)
        else:
            city = self.city_emb(city.view(-1))

        z = torch.cat([g, tab, city], dim=1)
        return self.mlp(z)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

node_in_dim = dataset[0].x.size(1)
tab_dim = len(dataset.feature_cols)
num_cities = len(dataset.city_to_idx)

model = GraphTabModel(node_in_dim=node_in_dim, tab_dim=tab_dim, num_cities=num_cities).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

model.train()
for epoch in range(3):
    total_loss = 0.0
    for batch in loader:
        batch = batch.to(device)
        pred = model(batch)
        yb = batch.y.view(-1, 1)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * yb.size(0)
    avg_loss = total_loss / len(loader.dataset)
    print(f"Epoch {epoch+1}: train MSE = {avg_loss:.4f}")

Epoch 1: train MSE = 1440878883.7656
Epoch 2: train MSE = 1163579265.3864
Epoch 3: train MSE = 1015631429.4365


# Evaluation for nn tuning

In [12]:
model.eval()
pred_loader = DataLoader(dataset, batch_size=2048, shuffle=False, num_workers=0, pin_memory=True)

all_pred = []
with torch.inference_mode():
    for batch in pred_loader:
        batch = batch.to(device)
        pred = model(batch).view(-1).detach().cpu().numpy()
        all_pred.append(pred)

filtered["y_pred"] = np.concatenate(all_pred)

C:\Users\kdgki\AppData\Local\Temp\ipykernel_37764\876383157.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered["y_pred"] = np.concatenate(all_pred)


In [13]:
filtered

,ur20,area,isoperi,solidity,rectan,pop_den,geom,city,graph,y_pred
0,U,16200.825,0.697,1.000,0.956,28616.003,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.2087, 0.7334, 1.6551, 1.41...",27773.201172
1,U,24292.633,0.609,1.000,0.949,8742.396,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.0095, 0.8443, 1.7148, 1.66...",28335.074219
2,U,19628.887,0.663,1.000,0.964,10555.762,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.1794, 0.7782, 1.6553, 1.51...",27931.412109
3,U,15393.900,0.561,1.000,0.979,7739.343,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.9450, 0.2050, 1.5157, 0.54...",29916.201172
4,U,14438.490,0.654,1.000,0.903,80365.333,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.8181, -0.3691, 1.3325, 0.67...",28376.552734
...,...,...,...,...,...,...,...,...,...,...
122510,U,1207.169,0.099,0.987,0.917,0.000,0106000020E6100000010000000103000000010000000A...,nyc,"[(x, [tensor([-2.9138, -0.3084, 1.9168, 0.19...",19507.916016
122511,U,26240.455,0.478,0.983,0.804,6415.748,0106000020E61000000100000001030000000100000007...,nyc,"[(x, [tensor([-1.0040, 0.0434, 3.1118, 0.41...",24177.544922
122512,U,26385.473,0.645,0.972,0.890,8638.202,0106000020E6100000010000000103000000010000000D...,nyc,"[(x, [tensor([-0.4232, -0.0432, 2.8995, 0.11...",16022.412109
122513,U,12150.579,0.616,1.000,0.979,9378.609,0106000020E61000000100000001030000000100000005...,nyc,"[(x, [tensor([-0.8584, 0.2414, 1.5201, 0.60...",28896.937500


In [14]:
import numpy as np

y_true = filtered["pop_den"].to_numpy(dtype=float)
y_pred = filtered["y_pred"].to_numpy(dtype=float)

ss_res = np.sum((y_true - y_pred) ** 2)
ss_tot = np.sum((y_true - y_true.mean()) ** 2)

r2 = 1.0 - ss_res / (ss_tot + 1e-12)

print("R2:", r2)


R2: 0.08332613003669509
